# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [14]:
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [15]:
training_datafile = "D:/PHD_DATA/Video_01-03-2021/prepared-data/single_train.npy"
training_rainfallfile = "D:/PHD_DATA/Video_01-03-2021/prepared-data/expected_train_standardized.npy"

In [16]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile).T

In [17]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [18]:
def apply_augmentation(raw_rain, raw_vids):
    vids = np.copy(raw_vids)
    vids = vids + np.random.normal(0, 1, vids.shape)
    return np.concatenate((raw_vids, vids), axis=0), np.concatenate((raw_rain, raw_rain), axis=0)

APPLY_AUGMENTATION = True

if APPLY_AUGMENTATION:
    training_videos, training_rainfall = apply_augmentation(training_rainfall, training_videos)

In [19]:
print(training_videos.shape)
print(training_rainfall.shape)

(408, 61, 121, 2)
(408, 13)


## 2. Model Definition

Next, a CNN model architecture is defined.

In [20]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(128, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(128, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(128, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [21]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_01-03-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(
        xdata,
        ydata,
        batch_size=32,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=150
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [22]:
# Flexible parameters
learning_rate = 0.0001

In [23]:
final_errors = {}
# Run model for each region
for ridx, r in enumerate(REGIONS):
    final_errors[r] = {}
    # Run each model multiple times
    for i in range(0, 3):
        run_name = "MF_{}_{}".format(r, int(time.time()))
        tb = TensorBoard(log_dir="D:/PHD_DATA/Video_01-03-2021/logs/{}".format(run_name))
        model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
        history = train_model(run_name, tb, model, training_videos, training_rainfall[:, ridx],
                              models_folder="D:/PHD_DATA/Video_01-03-2021/models/")
        final_errors[r][run_name] = history.history["val_loss"][-1]

Epoch 1/150
9/9 [==============================] - 13s 1s/step - loss: 0.0766 - val_loss: 0.0385
Epoch 2/150
9/9 [==============================] - 9s 989ms/step - loss: 0.0376 - val_loss: 0.0353
Epoch 3/150
9/9 [==============================] - 8s 930ms/step - loss: 0.0281 - val_loss: 0.0357
Epoch 4/150
9/9 [==============================] - 8s 885ms/step - loss: 0.0240 - val_loss: 0.0359
Epoch 5/150
9/9 [==============================] - 8s 864ms/step - loss: 0.0211 - val_loss: 0.0333
Epoch 6/150
9/9 [==============================] - 8s 890ms/step - loss: 0.0205 - val_loss: 0.0324
Epoch 7/150
9/9 [==============================] - 8s 895ms/step - loss: 0.0209 - val_loss: 0.0314
Epoch 8/150
9/9 [==============================] - 7s 837ms/step - loss: 0.0183 - val_loss: 0.0316
Epoch 9/150
9/9 [==============================] - 8s 861ms/step - loss: 0.0181 - val_loss: 0.0317
Epoch 10/150
9/9 [==============================] - 8s 862ms/step - loss: 0.0176 - val_loss: 0.0313
Epoch 11/15

Epoch 15/150
9/9 [==============================] - 8s 849ms/step - loss: 0.0211 - val_loss: 0.0339
Epoch 16/150
9/9 [==============================] - 8s 854ms/step - loss: 0.0190 - val_loss: 0.0327
Epoch 17/150
9/9 [==============================] - 8s 935ms/step - loss: 0.0182 - val_loss: 0.0336
Epoch 18/150
9/9 [==============================] - 8s 852ms/step - loss: 0.0178 - val_loss: 0.0317
Epoch 19/150
9/9 [==============================] - 7s 842ms/step - loss: 0.0137 - val_loss: 0.0320
Epoch 20/150
9/9 [==============================] - 8s 842ms/step - loss: 0.0171 - val_loss: 0.0312
Epoch 21/150
9/9 [==============================] - 8s 847ms/step - loss: 0.0166 - val_loss: 0.0333
Epoch 22/150
9/9 [==============================] - 8s 856ms/step - loss: 0.0159 - val_loss: 0.0311
Epoch 23/150
9/9 [==============================] - 8s 849ms/step - loss: 0.0162 - val_loss: 0.0325
Epoch 24/150
9/9 [==============================] - 8s 859ms/step - loss: 0.0200 - val_loss: 0.0308


9/9 [==============================] - 6s 730ms/step - loss: 0.0168 - val_loss: 0.0341
Epoch 00029: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_01-03-2021/models/MF_WS_1614764985.mdl\assets
Epoch 1/150
9/9 [==============================] - 9s 921ms/step - loss: 0.0875 - val_loss: 0.0467
Epoch 2/150
9/9 [==============================] - 6s 733ms/step - loss: 0.0386 - val_loss: 0.0441
Epoch 3/150
9/9 [==============================] - 7s 734ms/step - loss: 0.0359 - val_loss: 0.0677
Epoch 4/150
9/9 [==============================] - 6s 717ms/step - loss: 0.0353 - val_loss: 0.0420
Epoch 5/150
9/9 [==============================] - 6s 697ms/step - loss: 0.0303 - val_loss: 0.0384
Epoch 6/150
9/9 [==============================] - 6s 682ms/step - loss: 0.0273 - val_loss: 0.0451
Epoch 7/150
9/9 [==============================] - 6s 698ms/step - loss: 0.0297 - val_loss: 0.0471
Epoch 8/150
9/9 [==============================] - 6s 706ms/step - loss: 0.0288 - val_loss: 0

9/9 [==============================] - 6s 679ms/step - loss: 0.0290 - val_loss: 0.0473
Epoch 11/150
9/9 [==============================] - 6s 679ms/step - loss: 0.0307 - val_loss: 0.0467
Epoch 00011: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_01-03-2021/models/MF_EM_1614765663.mdl\assets
Epoch 1/150
9/9 [==============================] - 8s 878ms/step - loss: 0.0803 - val_loss: 0.0556
Epoch 2/150
9/9 [==============================] - 6s 700ms/step - loss: 0.0703 - val_loss: 0.0769
Epoch 3/150
9/9 [==============================] - 6s 699ms/step - loss: 0.0448 - val_loss: 0.0521
Epoch 4/150
9/9 [==============================] - 7s 741ms/step - loss: 0.0461 - val_loss: 0.0495
Epoch 5/150
9/9 [==============================] - 7s 741ms/step - loss: 0.0349 - val_loss: 0.0523
Epoch 6/150
9/9 [==============================] - 6s 716ms/step - loss: 0.0334 - val_loss: 0.0528
Epoch 7/150
9/9 [==============================] - 6s 704ms/step - loss: 0.0347 - val_loss: 

9/9 [==============================] - 6s 665ms/step - loss: 0.0504 - val_loss: 0.0565
Epoch 5/150
9/9 [==============================] - 6s 669ms/step - loss: 0.0385 - val_loss: 0.0569
Epoch 6/150
9/9 [==============================] - 6s 687ms/step - loss: 0.0409 - val_loss: 0.0481
Epoch 7/150
9/9 [==============================] - 6s 722ms/step - loss: 0.0413 - val_loss: 0.0453
Epoch 8/150
9/9 [==============================] - 7s 785ms/step - loss: 0.0353 - val_loss: 0.0440
Epoch 9/150
9/9 [==============================] - 6s 698ms/step - loss: 0.0391 - val_loss: 0.0469
Epoch 10/150
9/9 [==============================] - 6s 680ms/step - loss: 0.0356 - val_loss: 0.0428
Epoch 11/150
9/9 [==============================] - 6s 671ms/step - loss: 0.0342 - val_loss: 0.0508
Epoch 12/150
9/9 [==============================] - 6s 665ms/step - loss: 0.0341 - val_loss: 0.0439
Epoch 13/150
9/9 [==============================] - 6s 687ms/step - loss: 0.0289 - val_loss: 0.0480
Epoch 14/150
9/9 [

9/9 [==============================] - 7s 787ms/step - loss: 0.0337 - val_loss: 0.0433
Epoch 5/150
9/9 [==============================] - 6s 708ms/step - loss: 0.0265 - val_loss: 0.0404
Epoch 6/150
9/9 [==============================] - 6s 701ms/step - loss: 0.0261 - val_loss: 0.0392
Epoch 7/150
9/9 [==============================] - 6s 664ms/step - loss: 0.0257 - val_loss: 0.0386
Epoch 8/150
9/9 [==============================] - 6s 678ms/step - loss: 0.0235 - val_loss: 0.0382
Epoch 9/150
9/9 [==============================] - 6s 714ms/step - loss: 0.0201 - val_loss: 0.0392
Epoch 10/150
9/9 [==============================] - 6s 723ms/step - loss: 0.0225 - val_loss: 0.0388
Epoch 11/150
9/9 [==============================] - 6s 714ms/step - loss: 0.0178 - val_loss: 0.0397
Epoch 12/150
9/9 [==============================] - 6s 710ms/step - loss: 0.0206 - val_loss: 0.0372
Epoch 13/150
9/9 [==============================] - 6s 676ms/step - loss: 0.0197 - val_loss: 0.0374
Epoch 14/150
9/9 [

9/9 [==============================] - 11s 1s/step - loss: 0.0352 - val_loss: 0.0328
Epoch 3/150
9/9 [==============================] - 11s 1s/step - loss: 0.0246 - val_loss: 0.0337
Epoch 4/150
9/9 [==============================] - 11s 1s/step - loss: 0.0262 - val_loss: 0.0324
Epoch 5/150
9/9 [==============================] - 11s 1s/step - loss: 0.0208 - val_loss: 0.0305
Epoch 6/150
9/9 [==============================] - 11s 1s/step - loss: 0.0195 - val_loss: 0.0304
Epoch 7/150
9/9 [==============================] - 11s 1s/step - loss: 0.0175 - val_loss: 0.0285
Epoch 8/150
9/9 [==============================] - 11s 1s/step - loss: 0.0163 - val_loss: 0.0293
Epoch 9/150
9/9 [==============================] - 11s 1s/step - loss: 0.0160 - val_loss: 0.0290
Epoch 10/150
9/9 [==============================] - 11s 1s/step - loss: 0.0171 - val_loss: 0.0293
Epoch 11/150
9/9 [==============================] - 11s 1s/step - loss: 0.0146 - val_loss: 0.0315
Epoch 12/150
9/9 [======================

9/9 [==============================] - 11s 1s/step - loss: 0.0302 - val_loss: 0.0472
Epoch 7/150
9/9 [==============================] - 10s 1s/step - loss: 0.0288 - val_loss: 0.0355
Epoch 8/150
9/9 [==============================] - 11s 1s/step - loss: 0.0269 - val_loss: 0.0373
Epoch 9/150
9/9 [==============================] - 11s 1s/step - loss: 0.0267 - val_loss: 0.0357
Epoch 10/150
9/9 [==============================] - 11s 1s/step - loss: 0.0264 - val_loss: 0.0367
Epoch 11/150
9/9 [==============================] - 11s 1s/step - loss: 0.0245 - val_loss: 0.0376
Epoch 12/150
9/9 [==============================] - 11s 1s/step - loss: 0.0250 - val_loss: 0.0354
Epoch 13/150
9/9 [==============================] - 9s 1s/step - loss: 0.0268 - val_loss: 0.0378
Epoch 14/150
9/9 [==============================] - 8s 972ms/step - loss: 0.0223 - val_loss: 0.0357
Epoch 15/150
9/9 [==============================] - 9s 1s/step - loss: 0.0206 - val_loss: 0.0349
Epoch 16/150
9/9 [==================

Epoch 1/150
9/9 [==============================] - 13s 1s/step - loss: 0.0794 - val_loss: 0.0467
Epoch 2/150
9/9 [==============================] - 8s 921ms/step - loss: 0.0438 - val_loss: 0.0495
Epoch 3/150
9/9 [==============================] - 8s 949ms/step - loss: 0.0337 - val_loss: 0.0432
Epoch 4/150
9/9 [==============================] - 8s 973ms/step - loss: 0.0370 - val_loss: 0.0437
Epoch 5/150
9/9 [==============================] - 8s 933ms/step - loss: 0.0260 - val_loss: 0.0449
Epoch 6/150
9/9 [==============================] - 8s 947ms/step - loss: 0.0268 - val_loss: 0.0423
Epoch 7/150
9/9 [==============================] - 9s 996ms/step - loss: 0.0232 - val_loss: 0.0467
Epoch 8/150
9/9 [==============================] - 9s 1s/step - loss: 0.0224 - val_loss: 0.0438
Epoch 9/150
9/9 [==============================] - 8s 928ms/step - loss: 0.0255 - val_loss: 0.0395
Epoch 10/150
9/9 [==============================] - 8s 902ms/step - loss: 0.0216 - val_loss: 0.0375
Epoch 11/150
9

9/9 [==============================] - 8s 937ms/step - loss: 0.0263 - val_loss: 0.0427
Epoch 6/150
9/9 [==============================] - 8s 931ms/step - loss: 0.0255 - val_loss: 0.0394
Epoch 00006: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_01-03-2021/models/MF_YH_1614769661.mdl\assets
Epoch 1/150
9/9 [==============================] - 13s 1s/step - loss: 0.1236 - val_loss: 0.1350
Epoch 2/150
9/9 [==============================] - 8s 912ms/step - loss: 0.1317 - val_loss: 0.1350
Epoch 3/150
9/9 [==============================] - 8s 970ms/step - loss: 0.1292 - val_loss: 0.1350
Epoch 4/150
9/9 [==============================] - 8s 919ms/step - loss: 0.1274 - val_loss: 0.1350
Epoch 5/150
9/9 [==============================] - 8s 902ms/step - loss: 0.1130 - val_loss: 0.1350
Epoch 6/150
9/9 [==============================] - 9s 1s/step - loss: 0.1241 - val_loss: 0.1350
Epoch 00006: early stopping
INFO:tensorflow:Assets written to: D:/PHD_DATA/Video_01-03-2021/models

In [24]:
final_errors

{'ES': {'MF_ES_1614763873': 0.02989770844578743,
  'MF_ES_1614764116': 0.14216065406799316,
  'MF_ES_1614764178': 0.028072737157344818},
 'NS': {'MF_NS_1614764295': 0.04414105787873268,
  'MF_NS_1614764461': 0.026041464880108833,
  'MF_NS_1614764827': 0.03956743702292442},
 'WS': {'MF_WS_1614764985': 0.03405334800481796,
  'MF_WS_1614765222': 0.03302641585469246,
  'MF_WS_1614765515': 0.041163988411426544},
 'EM': {'MF_EM_1614765565': 0.04626349359750748,
  'MF_EM_1614765663': 0.04674812778830528,
  'MF_EM_1614765736': 0.04323173686861992},
 'EE': {'MF_EE_1614765890': 0.05156933516263962,
  'MF_EE_1614766042': 0.04129122197628021,
  'MF_EE_1614766216': 0.04178684204816818},
 'LD': {'MF_LD_1614766428': 0.036041196435689926,
  'MF_LD_1614766522': 0.033114008605480194,
  'MF_LD_1614766668': 0.033496398478746414},
 'NEE': {'MF_NEE_1614766711': 0.03604914993047714,
  'MF_NEE_1614766914': 0.03232000023126602,
  'MF_NEE_1614767121': 0.036094535142183304},
 'NWE': {'MF_NWE_1614767254': 0.02958